XOR problem
논리게이트 AND, OR, NAND, XOR

and = x1과2에가 동시에 1이 들어갈떄만 1을 출력    - 자동문
or  = 두개의 출력이 00일떄만 0을 출력
nand = 두개의 출력이 1일때 0을 출력               - 휴대폰 d램
xor = 두개의 출력이 0또는 1일때만 0을 출력        - 휴대폰 패턴암호

AND, OR, NAND, XOR 논리테이블(Logic Table)은 입력데이터 (x1, x2), 정답데이터 t (0 또는 1) 인 머신러닝 Training Data와 개념적으로 동일함
- 즉, 논리게이트는 손실함수로 cross-entropy를 이용해서 Logistic Regression (Classification)* 알고리즘으로 데이터를 분류 하고 결과를 예측 할 수 있음


2가지이상의 조합으로 이전 게이트의 모든 출력은 다음 gate의 입력으로 가져온다(완전연결계층 fc layer)

In [27]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

np.random.seed(0)

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad


def sigmoid(z):
    
    return 1 / (1+np.exp(-z))

In [28]:
class LogicGate :
    
    # constructor
    def __init__(self, xdata, tdata, learning_rate, iteration_count, cross_entropy=True):
            
        # 가중치 W 형상을 자동으로 구하기 위해 입력데이터가 vector 인지,
        # 아니면 matrix 인지 체크 후, 
        # self.xdata 는 무조건 matrix 로 만들어 주면 코드 일관성이 있음
        
        if xdata.ndim == 1:    # vector 차원으로 들어오면 행렬로 바꿔줌
            self.xdata = xdata.reshape(len(xdata), 1)
            self.tdata = xdata.reshape(len(tdata), 1)
            
        elif xdata.ndim == 2:  # matrix 행렬로 들어옴
            self.xdata = xdata
            self.tdata = tdata
        
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        
        self.W = np.random.rand(self.xdata.shape[1], 1) 
        self.b = np.random.rand(1)
        
        self.cross_entropy = cross_entropy
        
        self.loss_val_list = []
        # 빈 손실함수 리스트를 생성, 리턴값을 대기
        
        print("SimpleClassificationTest Object is created")
        
        if cross_entropy == True:
            print('loss function is set to cross entropy')
        else:
            print('loss function is set to MSE')
        
        
    # obtain current W and current b
    def getW_b(self):
        
        return self.W, self.b
    
    
    # loss function
    def loss_func(self):
    
        z = np.dot(self.xdata, self.W) + self.b
        
        y = sigmoid(z)
    
        if self.cross_entropy == True:
            
            delta = 1e-7    # log 무한대 발산 방지
            
            # cross-entropy 
            return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) ) 
        
        else:  # loss function == MSE
            
            return ( np.sum((self.tdata-y)**2) ) / ( len(self.xdata) )
            
    
    # display current error value
    def error_val(self):
        
        z = np.dot(self.xdata, self.W) + self.b
        
        y = sigmoid(z)
    
        if self.cross_entropy == True:
            
            delta = 1e-7    # log 무한대 발산 방지
                
            # cross-entropy 
            return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) ) 
        
        else:  # loss function == MSE
            
            return ( np.sum((self.tdata-y)**2) ) / ( len(self.xdata) )
    
    
    # predict method
    # 학습을 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
    # 입력변수 x : numpy type
    def predict(self, test_data):
    
        z = np.dot(test_data, self.W) + self.b
        y = sigmoid(z)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    
    def display_loss_val(self):
        
        if len(self.loss_val_list) == 0:
            print('loss_val is currently empty')
            return
        
        plt.title('Loss Value Trend')
        plt.ylabel('loss value')
        plt.grid()
        
        plt.plot(self.loss_val_list)
        plt.show()
        
    # train method
    def train(self):
    
        f = lambda x : self.loss_func()

        print("Initial error value = ", self.error_val() )

        start_time = datetime.now()
        
        for step in  range(self.iteration_count):  
    
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
    
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
    
            if (step % (int)(0.05*self.iteration_count) == 0):
            
                current_loss_val = self.error_val()                
                print("step = ", step, "error value = ", current_loss_val )                
                self.loss_val_list.append(current_loss_val)
                
                
        end_time = datetime.now()
        
        print("")
        print("Elapsed Time => ", end_time - start_time)

In [45]:
x_data = np.array( [ [0, 0], [0, 1], [1, 0], [1, 1] ] )
t_data = np.array( [0, 0, 0, 1] ).reshape(4, 1)
print("x_data.shape = ", x_data.shape, ", t_data.shape = ", t_data.shape)

x_data.shape =  (4, 2) , t_data.shape =  (4, 1)


In [49]:
obj1 = LogicGate(x_data, t_data, 1e-4, 100001, False)

obj1.train()

SimpleClassificationTest Object is created
loss function is set to MSE
Initial error value =  0.48100219268940436
step =  0 error value =  0.4809982784069919
step =  5000 error value =  0.46070246718971897
step =  10000 error value =  0.43906616294770895
step =  15000 error value =  0.416342200904206
step =  20000 error value =  0.39291102264198385
step =  25000 error value =  0.36926922653876754
step =  30000 error value =  0.34598937354307896
step =  35000 error value =  0.32365371129063997
step =  40000 error value =  0.3027767529744471
step =  45000 error value =  0.2837394106164825
step =  50000 error value =  0.2667541675891112
step =  55000 error value =  0.2518676406051516
step =  60000 error value =  0.23899231651695646
step =  65000 error value =  0.22795165207840326
step =  70000 error value =  0.21852394915943027
step =  75000 error value =  0.21047653787121598
step =  80000 error value =  0.2035880085016708
step =  85000 error value =  0.197660075780879
step =  90000 error

In [50]:
test_data = np.array([0, 0])
(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[0.35097263] 0
